In [ ]:
import os
import cv2
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.model_selection import train_test_split
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Dropout

In [ ]:
dataset_path = "/content/drive/MyDrive/Vehicle Classification Dataset (VCset)"

IMG_SIZE = (224, 224)
BATCH_SIZE = 32


In [ ]:
datagen = ImageDataGenerator(
    rescale=1./255,          # Normalize pixel values
    validation_split=0.2,    # Split dataset into training (80%) and validation (20%)
    rotation_range=20,       # Randomly rotate images by up to 20 degrees
    width_shift_range=0.2,   # Randomly shift images horizontally by up to 20%
    height_shift_range=0.2,  # Randomly shift images vertically by up to 20%
    shear_range=0.2,         # Apply random shearing transformations
    zoom_range=0.2,          # Randomly zoom into images by up to 20%
    horizontal_flip=True     # Randomly flip images horizontally
)

# Create training and validation data generators
train_generator = datagen.flow_from_directory(
    dataset_path,
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='sparse',     # Use sparse categorical labels (numerical)
    subset='training'        # Training subset
)

val_generator = datagen.flow_from_directory(
    dataset_path,
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='sparse',     # Validation subset
    subset='validation'
)


Found 8000 images belonging to 5 classes.
Found 2000 images belonging to 5 classes.


In [ ]:
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Dropout

# Load MobileNetV2 as the base model (pretrained on ImageNet)
base_model = MobileNetV2(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
base_model.trainable = False  # Freeze base model layers

# Add custom layers for classification
model = Sequential([
    base_model,
    GlobalAveragePooling2D(),
    Dense(128, activation='relu'),
    Dropout(0.5),            # Dropout for regularization
    Dense(5, activation='softmax')  # Output layer (5 classes)
])

# Compile the model with Adam optimizer and sparse categorical crossentropy loss
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

model.summary()

9406464/9406464 ━━━━━━━━━━━━━━━━━━━━ 1s 0us/step


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ mobilenetv2_1.00_224 (Functional)    │ (None, 7, 7, 1280)          │       2,257,984 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ global_average_pooling2d             │ (None, 1280)                │               0 │
│ (GlobalAveragePooling2D)             │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ (None, 128)                 │         163,968 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout (Dropout)                    │ (None, 128)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ (None, 5)                   │             645 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 2,422,597 (9.24 MB)

 Trainable params: 164,613 (643.02 KB)

 Non-trainable params: 2,257,984 (8.61 MB)

In [ ]:
history = model.fit(
    train_generator,
    validation_data=val_generator,
    epochs=10,                # Number of epochs for training
    steps_per_epoch=len(train_generator),
    validation_steps=len(val_generator)
)


/usr/local/lib/python3.11/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/10
250/250 ━━━━━━━━━━━━━━━━━━━━ 2111s 8s/step - accuracy: 0.5752 - loss: 1.0846 - val_accuracy: 0.7360 - val_loss: 0.6669
Epoch 2/10
250/250 ━━━━━━━━━━━━━━━━━━━━ 647s 3s/step - accuracy: 0.7364 - loss: 0.6999 - val_accuracy: 0.7580 - val_loss: 0.6498
Epoch 3/10
250/250 ━━━━━━━━━━━━━━━━━━━━ 682s 3s/step - accuracy: 0.7438 - loss: 0.6468 - val_accuracy: 0.7875 - val_loss: 0.5456
Epoch 4/10
250/250 ━━━━━━━━━━━━━━━━━━━━ 675s 3s/step - accuracy: 0.7744 - loss: 0.5787 - val_accuracy: 0.7885 - val_loss: 0.5495
Epoch 5/10
250/250 ━━━━━━━━━━━━━━━━━━━━ 663s 2s/step - accuracy: 0.7908 - loss: 0.5649 - val_accuracy: 0.7885 - val_loss: 0.5558
Epoch 6/10
250/250 ━━━━━━━━━━━━━━━━━━━━ 622s 2s/step - accuracy: 0.7937 - loss: 0.5466 - val_accuracy: 0.8110 - val_loss: 0.5056
Epoch 7/10
250/250 ━━━━━━━━━━━━━━━━━━━━ 703s 3s/step - accuracy: 0.7982 - loss: 0.5210 - val_accuracy: 0.8370 - val_loss: 0.4714
Epoch 8/10
250/250 ━━━━━━━━━━━━━━━━━━━━ 631s 3s/step - accuracy: 0.8043 - loss: 0.5159 - val_acc

In [ ]:
model.save("/content/drive/MyDrive/vehicle_classification_model_augmented.h5")


In [ ]:
model = load_model("/content/drive/MyDrive/vehicle_classification_model_augmented.h5")
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])


In [ ]:
val_loss, val_accuracy = model.evaluate(val_generator)
print(f"Validation Loss: {val_loss}, Validation Accuracy: {val_accuracy}")


63/63 ━━━━━━━━━━━━━━━━━━━━ 126s 2s/step - accuracy: 0.8378 - loss: 0.4350
Validation Loss: 0.43780818581581116, Validation Accuracy: 0.8364999890327454


In [ ]:
model = load_model("/content/drive/MyDrive/vehicle_classification_model_augmented.h5")
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

In [ ]:
history = model.fit(
    train_generator,
    validation_data=val_generator,
    epochs=10,                # Number of epochs for training
    steps_per_epoch=len(train_generator),
    validation_steps=len(val_generator)
)

/usr/local/lib/python3.11/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/10
250/250 ━━━━━━━━━━━━━━━━━━━━ 2809s 11s/step - accuracy: 0.8265 - loss: 0.4872 - val_accuracy: 0.8215 - val_loss: 0.4769
Epoch 2/10
250/250 ━━━━━━━━━━━━━━━━━━━━ 636s 2s/step - accuracy: 0.8242 - loss: 0.4610 - val_accuracy: 0.8430 - val_loss: 0.4272
Epoch 3/10
250/250 ━━━━━━━━━━━━━━━━━━━━ 614s 2s/step - accuracy: 0.8304 - loss: 0.4640 - val_accuracy: 0.8345 - val_loss: 0.4472
Epoch 4/10
250/250 ━━━━━━━━━━━━━━━━━━━━ 588s 2s/step - accuracy: 0.8470 - loss: 0.4385 - val_accuracy: 0.8415 - val_loss: 0.4432
Epoch 5/10
250/250 ━━━━━━━━━━━━━━━━━━━━ 592s 2s/step - accuracy: 0.8311 - loss: 0.4495 - val_accuracy: 0.8420 - val_loss: 0.4234
Epoch 6/10
250/250 ━━━━━━━━━━━━━━━━━━━━ 588s 2s/step - accuracy: 0.8348 - loss: 0.4338 - val_accuracy: 0.8400 - val_loss: 0.4395
Epoch 7/10
250/250 ━━━━━━━━━━━━━━━━━━━━ 610s 2s/step - accuracy: 0.8360 - loss: 0.4446 - val_accuracy: 0.8620 - val_loss: 0.3887
Epoch 8/10
250/250 ━━━━━━━━━━━━━━━━━━━━ 624s 2s/step - accuracy: 0.8453 - loss: 0.4140 - val_ac

In [ ]:
model.save("/content/drive/MyDrive/vehicle_classification_model_augmented1.h5")

In [ ]:
model = load_model("/content/drive/MyDrive/vehicle_classification_model_augmented1.h5")
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

In [ ]:
val_loss, val_accuracy = model.evaluate(val_generator)
print(f"Validation Loss: {val_loss}, Validation Accuracy: {val_accuracy}")

In [ ]:
model = load_model("/content/drive/MyDrive/vehicle_classification_model_augmented1.h5")
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

In [ ]:
history = model.fit(
    train_generator,
    validation_data=val_generator,
    epochs=15,                # Number of epochs for training
    steps_per_epoch=len(train_generator),
    validation_steps=len(val_generator)
)

/usr/local/lib/python3.11/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/15
250/250 ━━━━━━━━━━━━━━━━━━━━ 4001s 16s/step - accuracy: 0.8452 - loss: 0.4047 - val_accuracy: 0.8470 - val_loss: 0.4364
Epoch 2/15
250/250 ━━━━━━━━━━━━━━━━━━━━ 652s 2s/step - accuracy: 0.8506 - loss: 0.4089 - val_accuracy: 0.8335 - val_loss: 0.4953
Epoch 3/15
250/250 ━━━━━━━━━━━━━━━━━━━━ 625s 2s/step - accuracy: 0.8594 - loss: 0.3770 - val_accuracy: 0.8340 - val_loss: 0.5250
Epoch 4/15
250/250 ━━━━━━━━━━━━━━━━━━━━ 611s 2s/step - accuracy: 0.8520 - loss: 0.4002 - val_accuracy: 0.8515 - val_loss: 0.4540
Epoch 5/15
250/250 ━━━━━━━━━━━━━━━━━━━━ 611s 2s/step - accuracy: 0.8664 - loss: 0.3872 - val_accuracy: 0.8480 - val_loss: 0.4257
Epoch 6/15
250/250 ━━━━━━━━━━━━━━━━━━━━ 618s 2s/step - accuracy: 0.8589 - loss: 0.3851 - val_accuracy: 0.8345 - val_loss: 0.4583
Epoch 7/15
250/250 ━━━━━━━━━━━━━━━━━━━━ 612s 2s/step - accuracy: 0.8640 - loss: 0.3638 - val_accuracy: 0.8620 - val_loss: 0.4115
Epoch 8/15
250/250 ━━━━━━━━━━━━━━━━━━━━ 609s 2s/step - accuracy: 0.8696 - loss: 0.3506 - val_ac

In [ ]:
model.save("/content/drive/MyDrive/vehicle_classification_model_augmented2.keras")

In [ ]:
model = load_model("/content/drive/MyDrive/vehicle_classification_model_augmented2.h5")
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

In [ ]:
val_loss, val_accuracy = model.evaluate(val_generator)
print(f"Validation Loss: {val_loss}, Validation Accuracy: {val_accuracy}")

63/63 ━━━━━━━━━━━━━━━━━━━━ 124s 2s/step - accuracy: 0.8616 - loss: 0.4175
Validation Loss: 0.39954569935798645, Validation Accuracy: 0.8669999837875366


In [ ]:
model = load_model("/content/drive/MyDrive/vehicle_classification_model_augmented2.h5")
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

In [ ]:
history = model.fit(
    train_generator,
    validation_data=val_generator,
    epochs=20,                # Number of epochs for training
    steps_per_epoch=len(train_generator),
    validation_steps=len(val_generator)
)

/usr/local/lib/python3.11/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/20
250/250 ━━━━━━━━━━━━━━━━━━━━ 6711s 27s/step - accuracy: 0.8647 - loss: 0.3623 - val_accuracy: 0.8510 - val_loss: 0.4094
Epoch 2/20
250/250 ━━━━━━━━━━━━━━━━━━━━ 564s 2s/step - accuracy: 0.8726 - loss: 0.3424 - val_accuracy: 0.8550 - val_loss: 0.3956
Epoch 3/20
250/250 ━━━━━━━━━━━━━━━━━━━━ 595s 2s/step - accuracy: 0.8742 - loss: 0.3464 - val_accuracy: 0.8555 - val_loss: 0.4508
Epoch 4/20
250/250 ━━━━━━━━━━━━━━━━━━━━ 564s 2s/step - accuracy: 0.8712 - loss: 0.3493 - val_accuracy: 0.8520 - val_loss: 0.4638
Epoch 5/20
250/250 ━━━━━━━━━━━━━━━━━━━━ 586s 2s/step - accuracy: 0.8670 - loss: 0.3561 - val_accuracy: 0.8490 - val_loss: 0.4066
Epoch 6/20
250/250 ━━━━━━━━━━━━━━━━━━━━ 526s 2s/step - accuracy: 0.8701 - loss: 0.3373 - val_accuracy: 0.8715 - val_loss: 0.3701
Epoch 7/20
250/250 ━━━━━━━━━━━━━━━━━━━━ 574s 2s/step - accuracy: 0.8804 - loss: 0.3297 - val_accuracy: 0.8715 - val_loss: 0.3980
Epoch 8/20
250/250 ━━━━━━━━━━━━━━━━━━━━ 574s 2s/step - accuracy: 0.8777 - loss: 0.3332 - val_ac

In [ ]:
model.save("/content/drive/MyDrive/vehicle_classification_model_augmented3.keras")

In [ ]:
val_loss, val_accuracy = model.evaluate(val_generator)
print(f"Validation Loss: {val_loss}, Validation Accuracy: {val_accuracy}")

63/63 ━━━━━━━━━━━━━━━━━━━━ 105s 2s/step - accuracy: 0.8530 - loss: 0.4239
Validation Loss: 0.3912762701511383, Validation Accuracy: 0.8644999861717224
